<a href="https://colab.research.google.com/github/Dhruv11981/Traffic-Prediction-System/blob/main/Traffic_Prediction_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.8 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Sample Data (replace this with your actual dataset)
data = {
    'Weather': ['Sunny', 'Rainy', 'Sunny', 'Sunny', 'Rainy', 'Sunny'],
    'Traffic_State': ['Free', 'Congested', 'Free', 'Congested', 'Free', 'Free']
}
df = pd.DataFrame(data)

# Step 1: Markov Chain Model
# Define states
states = ['Free', 'Congested']
transition_matrix = np.array([[0.7, 0.3],  # From Free
                               [0.4, 0.6]]) # From Congested

def markov_predict(current_state, steps=1):
    state_index = states.index(current_state)
    current_probs = transition_matrix[state_index]
    next_state_index = np.random.choice(range(len(states)), p=current_probs)
    return states[next_state_index]

# Example usage of Markov Chain
current_state = 'Free'
predicted_states = [current_state]
for _ in range(5):  # Predicting the next 5 states
    next_state = markov_predict(current_state)
    predicted_states.append(next_state)
    current_state = next_state

print("Predicted Traffic States (Markov Chain):", predicted_states)

# Step 2: Bayesian Network Model
# Define the structure of the Bayesian Network
model = BayesianNetwork([('Weather', 'Traffic_State')])

# Define the Conditional Probability Tables (CPT) using TabularCPD
# Define the CPT for Weather
cpt_weather = TabularCPD(variable='Weather', variable_card=2, values=[[0.7], [0.3]],
                         state_names={'Weather': ['Sunny', 'Rainy']})

# Define the CPT for Traffic_State given Weather
cpt_traffic = TabularCPD(variable='Traffic_State', variable_card=2,
                         values=[[0.8, 0.5], [0.2, 0.5]],
                         evidence=['Weather'], evidence_card=[2],
                         state_names={'Weather': ['Sunny', 'Rainy'],
                                     'Traffic_State': ['Free', 'Congested']})


# Creating the Bayesian model with probabilities
model.add_cpds(
    cpt_weather,
    cpt_traffic
)

# Validate the model
assert model.check_model()

# Inference
infer = VariableElimination(model)
query_result = infer.query(variables=['Traffic_State'], evidence={'Weather': 'Rainy'})
print("Predicted Traffic State given Weather is Rainy (Bayesian Network):")
print(query_result)

Predicted Traffic States (Markov Chain): ['Free', 'Free', 'Free', 'Free', 'Free', 'Congested']
Predicted Traffic State given Weather is Rainy (Bayesian Network):
+--------------------------+----------------------+
| Traffic_State            |   phi(Traffic_State) |
+==========================+======================+
| Traffic_State(Free)      |               0.5000 |
+--------------------------+----------------------+
| Traffic_State(Congested) |               0.5000 |
+--------------------------+----------------------+
